In [1]:
import os
from openai import OpenAI
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List, Optional, Tuple, Union

import PyPDF2
import markdown
import html2text
import json
from tqdm import tqdm
import tiktoken
import re
from bs4 import BeautifulSoup
from IPython.display import display, Code, Markdown

# 导入 python-dotenv 来读取 .env 文件
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv()

True

In [2]:
# 从环境变量获取API密钥
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

# 检查环境变量是否正确加载
if not api_key:
    raise ValueError("请设置 OPENAI_API_KEY 环境变量")
if not base_url:
    raise ValueError("请设置 OPENAI_BASE_URL 环境变量")

print("环境变量加载成功！")
print(f"API密钥前缀: {api_key[:10]}...")
print(f"Base URL: {base_url}")

环境变量加载成功！
API密钥前缀: sk-proj-R_...
Base URL: https://ai.devtool.tech/proxy/v1


In [3]:
# 实例化客户端 - 使用环境变量
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

print("OpenAI 客户端创建成功！")

OpenAI 客户端创建成功！


In [4]:
# 环境变量已通过 .env 文件加载，无需手动设置
print("环境变量通过 .env 文件自动加载")

环境变量通过 .env 文件自动加载


In [5]:
# 调用 embedding API 获取文本的向量表示
response = client.embeddings.create(
    input="测试文本",  # 输入文本
    model="text-embedding-3-small"  # 选择 Embedding 模型
)

# 打印返回的embedding 向量
print(response.data[0].embedding)

# 获取embedding长度
len(response.data[0].embedding)

[0.0060808719135820866, 0.01937379501760006, 0.01978626847267151, -0.04902195185422897, 0.005962129216641188, -0.05264672636985779, -0.026648342609405518, -0.012949194759130478, 0.0021545535419136286, -0.02132367342710495, 0.017861388623714447, 0.0028310739435255527, -0.01738641783595085, 0.016573969274759293, 0.02018624357879162, 0.061296187341213226, -0.007674522697925568, -0.009818139486014843, -0.021473664790391922, 0.030473103746771812, 0.024323487654328346, 0.012936695478856564, 0.010143118910491467, -0.034522853791713715, -0.012192991562187672, -0.0307230893522501, -0.03832261636853218, -0.03187301754951477, 0.0637960284948349, -0.05794639512896538, -0.03424786776304245, -0.04184739664196968, 0.018648840487003326, -0.016311485320329666, -0.004121618811041117, 0.039397548884153366, 0.04609713330864906, -0.03932255506515503, 0.010399352759122849, -0.08414476364850998, -0.022611092776060104, 0.006393352057784796, 0.023748522624373436, 0.014511597342789173, 0.023311050608754158, 0.0

1536

In [6]:
class BaseEmbeddings:
    """
    向量化的基类，用于将文本转换为向量表示。不同的子类可以实现不同的向量获取方法。
    """
    def __init__(self, path: str, is_api: bool) -> None:
        """
        初始化基类。
        
        参数：
        path (str) - 如果是本地模型，path 表示模型路径；如果是API模式，path可以为空
        is_api (bool) - 表示是否使用API调用，如果为True表示通过API获取Embedding
        """
        self.path = path
        self.is_api = is_api
    
    def get_embedding(self, text: str, model: str) -> List[float]:
        """
        抽象方法，用于获取文本的向量表示，具体实现需要在子类中定义。
        
        参数：
        text (str) - 需要转换为向量的文本
        model (str) - 所使用的模型名称
        
        返回：
        list[float] - 文本的向量表示
        """
        raise NotImplementedError
    
    @staticmethod
    def cosine_similarity(vector1: List[float], vector2: List[float]) -> float:
        """
        计算两个向量之间的余弦相似度，用于衡量它们的相似程度。
        
        参数：
        vector1 (list[float]) - 第一个向量
        vector2 (list[float]) - 第二个向量
        
        返回：
        float - 余弦相似度值，范围从 -1 到 1，越接近 1 表示向量越相似
        """
        dot_product = np.dot(vector1, vector2)  # 向量点积
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)  # 向量的模
        if not magnitude:
            return 0
        return dot_product / magnitude  # 计算余弦相似度

In [7]:
class OpenAIEmbedding(BaseEmbeddings):
    """
    使用 OpenAI 的 Embedding API 来获取文本向量的类，继承自 BaseEmbeddings。
    """
    def __init__(self, path: str = '', is_api: bool = True) -> None:
        """
        初始化类，设置 OpenAI API 客户端，如果使用的是 API 调用。
        
        参数：
        path (str) - 本地模型的路径，使用API时可以为空
        is_api (bool) - 是否通过 API 获取 Embedding，默认为 True
        """
        super().__init__(path, is_api)
        if self.is_api:
            # 初始化 OpenAI API 客户端
            from openai import OpenAI
            self.client = OpenAI()
            self.client.api_key = os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
            self.client.base_url = os.getenv("OPENAI_BASE_URL")  # 从环境变量中获取 API 基础URL
    
    def get_embedding(self, text: str, model: str = "text-embedding-3-large") -> List[float]:
        """
        使用 OpenAI 的 Embedding API 获取文本的向量表示。
        
        参数：
        text (str) - 需要转化为向量的文本
        model (str) - 使用的 Embedding 模型名称，默认为 'text-embedding-3-large'
        
        返回：
        list[float] - 文本的向量表示
        """
        if self.is_api:
            # 去掉文本中的换行符，保证输入格式规范
            text = text.replace("\n", " ")
            # 调用 OpenAI API 获取文本的向量表示
            return self.client.embeddings.create(
                input=[text], 
                model=model).data[0].embedding
        else:
            raise NotImplementedError  # 如果不是 API 模式，这里未实现本地模型的处理

In [8]:
# 初始化 Embedding 模型
embedding_model = OpenAIEmbedding()

# 输入需要获取向量表示的文本
text = "这是一个示例文本，用于演示 OpenAI Embedding 的使用。"

# 获取文本的向量表示
embedding_vector = embedding_model.get_embedding(text, model="text-embedding-3-large")

print("文本的向量表示为：", embedding_vector)

文本的向量表示为： [0.02657606266438961, 0.013006975874304771, -0.0070852069184184074, -0.010484014637768269, 0.019530072808265686, 0.01330763939768076, -0.001928169745951891, 0.0353737436234951, 0.0009869614150375128, -0.026419194415211678, -0.02088959515094757, 0.034667838364839554, -0.004320407286286354, -0.04125629737973213, 0.00809831265360117, 0.00262263766489923, 0.03479856252670288, 0.02700744941830635, -0.017961392179131508, -0.004281190689653158, 0.010863112285733223, 0.016183553263545036, -0.020680436864495277, -0.008810755796730518, 0.04070725664496422, -0.021294837817549706, 0.013974328525364399, 0.04154388979077339, -0.007333581335842609, 0.023935450240969658, -0.0015613272553309798, -0.033099159598350525, -0.0015809357864782214, -0.0391908660531044, 0.0050034369342029095, 0.014667162671685219, 0.04483811557292938, 0.0616491436958313, -0.012444864958524704, 0.0036373778712004423, 0.04034123197197914, -0.018536575138568878, 0.0015507059870287776, -0.026732929050922394, -0.016562651

In [9]:
text1 = '我喜欢吃苹果'
text2 = "苹果是我最喜欢吃的水果"
text3 = "我喜欢用苹果手机"

vector1 = embedding_model.get_embedding(text1)
vector2 = embedding_model.get_embedding(text2)

similarity = OpenAIEmbedding.cosine_similarity(vector1, vector2)
print(f"两段文本的余弦相似度为: {similarity}")

两段文本的余弦相似度为: 0.7661800739633337


In [10]:
enc = tiktoken.get_encoding("cl100k_base")

In [11]:
class ReadFiles:
    """
    读取文件的类，用于从指定路径读取支持的文件类型（如 .txt、.md、.pdf）并进行内容分割。
    """

    def __init__(self, path: str) -> None:
        """
        初始化函数，设定要读取的文件路径，并获取该路径下所有符合要求的文件。
        :param path: 文件夹路径
        """
        self._path = path
        self.file_list = self.get_files()  # 获取文件列表

    def get_files(self):
        """
        遍历指定文件夹，获取支持的文件类型列表（txt, md, pdf）。
        :return: 文件路径列表
        """
        file_list = []
        for filepath, dirnames, filenames in os.walk(self._path):
            # os.walk 函数将递归遍历指定文件夹
            for filename in filenames:
                # 根据文件后缀筛选支持的文件类型
                if filename.endswith(".md"):
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith(".txt"):
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith(".pdf"):
                    file_list.append(os.path.join(filepath, filename))
        return file_list

    def get_content(self, max_token_len: int = 600, cover_content: int = 150):
        """
        读取文件内容并进行分割，将长文本切分为多个块。
        :param max_token_len: 每个文档片段的最大 Token 长度
        :param cover_content: 在每个片段之间重叠的 Token 长度
        :return: 切分后的文档片段列表
        """
        docs = []
        for file in self.file_list:
            content = self.read_file_content(file)  # 读取文件内容
            # 分割文档为多个小块
            chunk_content = self.get_chunk(content, max_token_len=max_token_len, cover_content=cover_content)
            docs.extend(chunk_content)
        return docs

    @classmethod
    def get_chunk(cls, text: str, max_token_len: int = 600, cover_content: int = 150):
        """
        将文档内容按最大 Token 长度进行切分。
        :param text: 文档内容
        :param max_token_len: 每个片段的最大 Token 长度
        :param cover_content: 重叠的内容长度
        :return: 切分后的文档片段列表
        """
        chunk_text = []
        curr_len = 0
        curr_chunk = ''
        token_len = max_token_len - cover_content
        lines = text.splitlines()  # 以换行符分割文本为行

        for line in lines:
            line = line.replace(' ', '')  # 去除空格
            line_len = len(enc.encode(line))  # 计算当前行的 Token 长度
            if line_len > max_token_len:
                # 如果单行长度超过限制，将其分割为多个片段
                num_chunks = (line_len + token_len - 1) // token_len
                for i in range(num_chunks):
                    start = i * token_len
                    end = start + token_len
                    # 防止跨单词分割
                    while not line[start:end].rstrip().isspace():
                        start += 1
                        end += 1
                        if start >= line_len:
                            break
                    curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                    chunk_text.append(curr_chunk)
                start = (num_chunks - 1) * token_len
                curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                chunk_text.append(curr_chunk)
            elif curr_len + line_len <= token_len:
                # 当前片段长度未超过限制时，继续累加
                curr_chunk += line + '\n'
                curr_len += line_len + 1
            else:
                chunk_text.append(curr_chunk)  # 保存当前片段
                curr_chunk = curr_chunk[-cover_content:] + line
                curr_len = line_len + cover_content

        if curr_chunk:
            chunk_text.append(curr_chunk)

        return chunk_text

    @classmethod
    def read_file_content(cls, file_path: str):
        """
        读取文件内容，根据文件类型选择不同的读取方式。
        :param file_path: 文件路径
        :return: 文件内容
        """
        if file_path.endswith('.pdf'):
            return cls.read_pdf(file_path)
        elif file_path.endswith('.md'):
            return cls.read_markdown(file_path)
        elif file_path.endswith('.txt'):
            return cls.read_text(file_path)
        else:
            raise ValueError("Unsupported file type")

    @classmethod
    def read_pdf(cls, file_path: str):
        """
        读取 PDF 文件内容。
        :param file_path: PDF 文件路径
        :return: PDF 文件中的文本内容
        """
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
            return text

    @classmethod
    def read_markdown(cls, file_path: str):
        """
        读取 Markdown 文件内容，并将其转换为纯文本。
        :param file_path: Markdown 文件路径
        :return: 纯文本内容
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            md_text = file.read()
            html_text = markdown.markdown(md_text)
            # 使用 BeautifulSoup 从 HTML 中提取纯文本
            soup = BeautifulSoup(html_text, 'html.parser')
            plain_text = soup.get_text()
            # 使用正则表达式移除网址链接
            text = re.sub(r'http\S+', '', plain_text) 
            return text

    @classmethod
    def read_text(cls, file_path: str):
        """
        读取普通文本文件内容。
        :param file_path: 文本文件路径
        :return: 文件内容
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()

In [25]:
# 初始化 ReadFiles 类，指定文件目录路径
file_reader = ReadFiles(path="./data")

# 获取目录下所有支持的文件类型
file_list = file_reader.get_files()
print("支持的文件列表：", file_list)

支持的文件列表： ['./data/【人工智能】AI构建者手册2025 | ICONIQ发布68页报告| AI原生公司 | AI赋能公司 | 代理工作流 | 基础设施 | 市场定价 | 团队结构 | 成本预算 | 内部效率 - YouTube.txt']


In [26]:
# 将文件内容读取并分块
document_chunks = file_reader.get_content(max_token_len=600, cover_content=150)
print("分块后的文档内容：", document_chunks)

分块后的文档内容： ['大家好，这里是最佳拍档，我是大飞。不知道大家有没有发现，最近两年AI领域的讨论已经从“要不要做AI”变成了“怎么把AI做好”。从2023年生成式AI爆发开始，到2025年的今天，越来越多的公司不再纠结于AI的可能性，而是埋头在产品开发、团队搭建、成本控制这些实际的问题上。最近，曾经作为管理硅谷众多科技巨头家办的ICONIQ资本，抓住了这个转折点的时机，发布了一份长达68页的报告，拆解了300家正在开发AI产品的软件公司的实战经验。无论是AI原生公司，还是所谓的AI赋能公司，相信它们的踩坑经验、成功做法，都能带给我们很多的启发。今天大飞就来给大家分享一下。\n\n', '问题上。最近，曾经作为管理硅谷众多科技巨头家办的ICONIQ资本，抓住了这个转折点的时机，发布了一份长达68页的报告，拆解了300家正在开发AI产品的软件公司的实战经验。无论是AI原生公司，还是所谓的AI赋能公司，相信它们的踩坑经验、成功做法，都能带给我们很多的启发。今天大飞就来给大家分享一下。\n\n要聊AI产品的开发，首先我们得搞清楚现在市面上的AI公司到底有哪些类型。根据报告的调研，主要分为两类，分别是AI原生公司和AI赋能公司。AI原生公司指的是核心产品或商业模式完全由AI驱动的公司，它们的价值几乎都来自于模型训练、推理和持续的学习，这类公司在调研中占了32%，它们的特点是产品的迭代速度非常快。报告里提到，只有1%的AI原生公司还停留在产品发布前的阶段，而AI赋能公司则有11%卡在这个阶段；更关键的是，47%的AI原生产品已经进入了规模化的阶段，也就是说产品已经验证了市场契合度，正在快速扩大用户群和基础设施。这背后可能的原因是，AI原生公司在团队构成、基础设施和融资模式上更有优势，能够更快的跳过试错阶段，而AI赋能公司往往需要把AI“嫁接”到现有的工作流中，难免就会遇到更多的阻碍。', '段；更关键的是，47%的AI原生产品已经进入了规模化的阶段，也就是说产品已经验证了市场契合度，正在快速扩大用户群和基础设施。这背后可能的原因是，AI原生公司在团队构成、基础设施和融资模式上更有优势，能够更快的跳过试错阶段，而AI赋能公司往往需要把AI“嫁接”到现有的工作流中，难免就会遇到更多的阻碍。AI赋能公司则分为两种，一种是在旗舰产品中嵌入AI功能，比如给现有的CRM系统加一个AI

In [31]:
len(document_chunks)

28

In [29]:
class VectorStore:
    def __init__(self, document: List[str] = None) -> None:
        """
        初始化向量存储类，存储文档和对应的向量表示。
        :param document: 文档列表，默认为空。
        """
        if document is None:
            document = []
        self.document = document  # 存储文档内容
        self.vectors = []  # 存储文档的向量表示
    
    def get_vector(self, EmbeddingModel: BaseEmbeddings) -> List[List[float]]:
        """
        使用传入的 Embedding 模型将文档向量化。
        :param EmbeddingModel: 传入的用于生成向量的模型（需继承 BaseEmbeddings 类）。
        :return: 返回文档对应的向量列表。
        """
        # 遍历所有文档，获取每个文档的向量表示
        self.vectors = [EmbeddingModel.get_embedding(doc) for doc in self.document]
        return self.vectors
    
    def persist(self, path: str = 'storage'):
        """
        将文档和对应的向量表示持久化到本地目录中，以便后续加载使用。
        :param path: 存储路径，默认为 'storage'。
        """
        if not os.path.exists(path):
            os.makedirs(path)  # 如果路径不存在，创建路径
        # 保存向量为 numpy 文件
        np.save(os.path.join(path, 'vectors.npy'), self.vectors)
        # 将文档内容存储到文本文件中
        with open(os.path.join(path, 'documents.txt'), 'w') as f:
            for doc in self.document:
                f.write(f"{doc}\n")
    
    def load_vector(self, path: str = 'storage'):
        """
        从本地加载之前保存的文档和向量数据。
        :param path: 存储路径，默认为 'storage'。
        """
        # 加载保存的向量数据
        self.vectors = np.load(os.path.join(path, 'vectors.npy')).tolist()
        # 加载文档内容
        with open(os.path.join(path, 'documents.txt'), 'r') as f:
            self.document = [line.strip() for line in f.readlines()]

    def get_similarity(self, vector1: List[float], vector2: List[float]) -> float:
        """
        计算两个向量的余弦相似度。
        :param vector1: 第一个向量。
        :param vector2: 第二个向量。
        :return: 返回两个向量的余弦相似度，范围从 -1 到 1。
        """
        dot_product = np.dot(vector1, vector2)
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)
        if not magnitude:
            return 0
        return dot_product / magnitude

    def query(self, query: str, EmbeddingModel: BaseEmbeddings, k: int = 1) -> List[str]:
        """
        根据用户的查询文本，检索最相关的文档片段。
        :param query: 用户的查询文本。
        :param EmbeddingModel: 用于将查询向量化的嵌入模型。
        :param k: 返回最相似的文档数量，默认为 1。
        :return: 返回最相似的文档列表。
        """
        # 将查询文本向量化
        query_vector = EmbeddingModel.get_embedding(query)
        # 计算查询向量与每个文档向量的相似度
        similarities = [self.get_similarity(query_vector, vector) for vector in self.vectors]
        # 获取相似度最高的 k 个文档索引
        top_k_indices = np.argsort(similarities)[-k:][::-1]
        # 返回对应的文档内容
        return [self.document[idx] for idx in top_k_indices]

In [30]:
# 创建向量数据库
vector_store = VectorStore(document=document_chunks)

# 使用 OpenAI Embedding 模型对文档进行向量化
embedding_model = OpenAIEmbedding()

# 获取文档向量并存储
vector_store.get_vector(embedding_model)

# 持久化存储到本地
vector_store.persist('storage')

In [ ]:
# 模拟用户查询
query = "AI公司分为哪两种类型？"
result = vector_store.query(query, embedding_model)

print("检索结果：", result)

检索结果： ['问题上。最近，曾经作为管理硅谷众多科技巨头家办的ICONIQ资本，抓住了这个转折点的时机，发布了一份长达68页的报告，拆解了300家正在开发AI产品的软件公司的实战经验。无论是AI原生公司，还是所谓的AI赋能公司，相信它们的踩坑经验、成功做法，都能带给我们很多的启发。今天大飞就来给大家分享一下。\n\n要聊AI产品的开发，首先我们得搞清楚现在市面上的AI公司到底有哪些类型。根据报告的调研，主要分为两类，分别是AI原生公司和AI赋能公司。AI原生公司指的是核心产品或商业模式完全由AI驱动的公司，它们的价值几乎都来自于模型训练、推理和持续的学习，这类公司在调研中占了32%，它们的特点是产品的迭代速度非常快。报告里提到，只有1%的AI原生公司还停留在产品发布前的阶段，而AI赋能公司则有11%卡在这个阶段；更关键的是，47%的AI原生产品已经进入了规模化的阶段，也就是说产品已经验证了市场契合度，正在快速扩大用户群和基础设施。这背后可能的原因是，AI原生公司在团队构成、基础设施和融资模式上更有优势，能够更快的跳过试错阶段，而AI赋能公司往往需要把AI“嫁接”到现有的工作流中，难免就会遇到更多的阻碍。']


In [36]:
class BaseModel:
    """
    基础模型类，作为所有模型的基类。
    包含一些通用的接口，如加载模型、生成回答等。
    """
    def __init__(self, path: str = '') -> None:
        self.path = path  # 用于存储模型文件的路径，默认为空。

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        """
        使用模型生成回答的抽象方法。
        :param prompt: 用户的提问内容
        :param history: 之前的对话历史（字典列表）
        :param content: 提供的上下文内容
        :return: 模型生成的答案
        """
        pass  # 具体的实现由子类提供

    def load_model(self):
        """
        加载模型的方法，通常用于本地模型。
        """
        pass  # 如果是 API 模型，可能不需要实现

In [37]:
class GPT4oChat(BaseModel):
    """
    基于 GPT-4o 模型的对话类，继承自 BaseModel。
    主要用于通过 OpenAI API 来生成对话回答。
    """
    def __init__(self, api_key: str, base_url: str = "https://ai.devtool.tech/proxy/v1") -> None:
        """
        初始化 GPT-4o 模型。
        :param api_key: OpenAI API 的密钥
        :param base_url: 用于访问 OpenAI API 的基础 URL，默认为代理 URL
        """
        super().__init__()
        self.client = OpenAI(api_key=api_key, base_url=base_url)  # 初始化 OpenAI 客户端

    def chat(self, prompt: str, history: List = [], content: str = '') -> str:
        """
        使用 GPT-4o 生成回答。
        :param prompt: 用户的提问
        :param history: 之前的对话历史（可选）
        :param content: 可参考的上下文信息（可选）
        :return: 生成的回答
        """
        # 构建包含问题和上下文的完整提示
        full_prompt = PROMPT_TEMPLATE['GPT4o_PROMPT_TEMPLATE'].format(question=prompt, context=content)

        # 调用 GPT-4o 模型进行推理
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",  # 使用 GPT-4o 小型模型
            messages=[
                {"role": "user", "content": full_prompt}
            ]
        )

        # 返回模型生成的第一个回答
        return response.choices[0].message.content

In [38]:
PROMPT_TEMPLATE = dict(
    GPT4o_PROMPT_TEMPLATE="""
    下面有一个或许与这个问题相关的参考段落，若你觉得参考段落能和问题相关，则先总结参考段落的内容。
    若你觉得参考段落和问题无关，则使用你自己的原始知识来回答用户的问题，并且总是使用中文来进行回答。
    问题: {question}
    可参考的上下文：
    ···
    {context}
    ···
    有用的回答:"""
)

In [39]:
chat = GPT4oChat(api_key=api_key)
question = 'AI公司分为哪两种类型？'
answer = chat.chat(question, [], result)


In [41]:
answer

'参考段落的内容总结：报告指出，目前市场上的AI公司主要分为两种类型：AI原生公司和AI赋能公司。AI原生公司是指其核心产品或商业模式完全由AI驱动，主要依靠模型训练、推理和持续学习，这类公司在报告中占32%，并且其产品迭代速度快。相较之下，AI赋能公司则较为依赖将AI技术整合到现有工作流中，面临更多的挑战，11%的AI赋能公司还处于产品发布前的阶段。\n\n回答问题：AI公司主要分为两种类型：AI原生公司和AI赋能公司。'

In [42]:
display(Markdown(answer))

参考段落的内容总结：报告指出，目前市场上的AI公司主要分为两种类型：AI原生公司和AI赋能公司。AI原生公司是指其核心产品或商业模式完全由AI驱动，主要依靠模型训练、推理和持续学习，这类公司在报告中占32%，并且其产品迭代速度快。相较之下，AI赋能公司则较为依赖将AI技术整合到现有工作流中，面临更多的挑战，11%的AI赋能公司还处于产品发布前的阶段。

回答问题：AI公司主要分为两种类型：AI原生公司和AI赋能公司。

In [40]:
# def run_mini_rag(question: str, knowledge_base_path: str, k: int = 1) -> str:
#     """
#     运行一个简化版的RAG项目。
    
#     :param question: 用户提出的问题
#     :param knowledge_base_path: 知识库的路径，包含文档的文件夹路径
#     :param api_key: OpenAI API密钥，用于调用GPT-4o模型
#     :param k: 返回与问题最相关的k个文档片段，默认为1
#     :return: 返回GPT-4o模型生成的回答
#     """
#     api_key = os.getenv("OPENAI_API_KEY")
#     # 1. 加载并切分文档
#     docs = ReadFiles(knowledge_base_path).get_content(max_token_len=600, cover_content=150)
#     vector = VectorStore(docs)

#     # 2. 使用 OpenAI Embedding 模型进行向量化
#     embedding = OpenAIEmbedding()
#     vector.get_vector(EmbeddingModel=embedding)

#     # 3. 将向量和文档保存到本地（可选）
#     vector.persist(path='storage')

#     # 4. 在数据库中检索最相关的文档片段
#     content = vector.query(question, EmbeddingModel=embedding, k=k)[0]

#     # 5. 使用 GPT-4o 生成答案
#     chat = GPT4oChat(api_key=api_key)
#     answer = chat.chat(question, [], content)
    
#     return answer